In [48]:
from selenium import webdriver
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from fake_useragent import UserAgent
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import re
from bs4 import BeautifulSoup
import requests
import os
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 100)
import pdfquery
from lxml import etree
import xml.etree.cElementTree as ET
import re
import sklearn.cluster
import distance
import string
import spacy
import nltk
from fuzzywuzzy import fuzz
from nltk.corpus import wordnet as wn
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from spacy.lang.en import English
from gensim import corpora
import gensim
import pickle
import os
from gensim.models.wrappers import LdaMallet
from textblob import TextBlob
import math

In [4]:
for vol in range(538,581,1):
    pdf_file = "D:\FHSnotes\Thesis\Hansard\Vol {vol}.pdf".format(vol=vol)
    pdf = pdfquery.PDFQuery(pdf_file)
    pdf.load() # This is the bit that takes a very long time
    pdf.tree.write(pdf_file[0:-4]+".xml", pretty_print=True, encoding="utf-8")

In [3]:
def combine(files):
    first = None
    for filename in files:
        data = ET.parse(filename).getroot()
        if first is None:
            first = data
        else:
            first.extend(data)
    if first is not None:
        f = open('D:\FHSnotes\Thesis\Hansard\Parl44.xml', 'wb')
        f.write(ET.tostring(first))
        f.close()
        # return et.tostring(first)

combine(('D:\FHSnotes\Thesis\Hansard\Parl44.xml','D:\FHSnotes\Thesis\Hansard\Vol 540.xml','D:\FHSnotes\Thesis\Hansard\Vol 541.xml','D:\FHSnotes\Thesis\Hansard\Vol 542.xml','D:\FHSnotes\Thesis\Hansard\Vol 543.xml','D:\FHSnotes\Thesis\Hansard\Vol 544.xml','D:\FHSnotes\Thesis\Hansard\Vol 545.xml','D:\FHSnotes\Thesis\Hansard\Vol 546.xml','D:\FHSnotes\Thesis\Hansard\Vol 547.xml','D:\FHSnotes\Thesis\Hansard\Vol 548.xml','D:\FHSnotes\Thesis\Hansard\Vol 549.xml','D:\FHSnotes\Thesis\Hansard\Vol 550.xml','D:\FHSnotes\Thesis\Hansard\Vol 551.xml','D:\FHSnotes\Thesis\Hansard\Vol 552.xml','D:\FHSnotes\Thesis\Hansard\Vol 553.xml','D:\FHSnotes\Thesis\Hansard\Vol 554.xml','D:\FHSnotes\Thesis\Hansard\Vol 555.xml','D:\FHSnotes\Thesis\Hansard\Vol 556.xml','D:\FHSnotes\Thesis\Hansard\Vol 557.xml'))

In [3]:
pdf_file = "D:\FHSnotes\Thesis\Hansard\Parl44.xml" 
exp = re.compile(r'<.*?>')
with open(pdf_file[0:-4]+".xml") as f:
    lines = ''.join(line for line in f.readlines())
    text_only = exp.sub('',lines).strip()

In [4]:
speakers_indices = [(m.start(0), m.end(0)) for m in re.finditer(r'[\r\n]+.{0,100}:', text_only)]

In [5]:
speakers = [text_only[a:b] for (a,b) in speakers_indices]
speakers

['\n     JOHN CARTER (Senior Government Whip):',
 '\n     Hon. K T WETERE (Western Maori):',
 '\n     [Subsequent authorised translation:',
 '\n     CLERK OF THE HOUSE:',
 '\n     Hon. PETER TAPSELL (Eastern Maori):',
 '\n     CLERK OF THE HOUSE:',
 '\n     WINSTON PETERS (Tauranga):',
 '\n     Rt Hon. J B Bolger:',
 '\n     WINSTON PETERS:',
 '\n     Rt Hon. J B Bolger:',
 '\n     WINSTON PETERS:',
 '\n     Rt Hon. Mike Moore:',
 '\n     WINSTON PETERS:',
 '\n     TAU HENARE (Northern Maori):',
 '\n     H V Ross Robertson:',
 '\n     Rt Hon. Don McKinnon:',
 '\n     Winston Peters:',
 '\n     Rt Hon. Jonathan Hunt:',
 '\n     Hon. Paul East:',
 '\n     CLERK OF THE HOUSE:',
 '\n     TAU HENARE:',
 '\n     CLERK OF THE HOUSE:',
 '\n     Hon. ROBIN GRAY (Clutha):',
 '\n     CLERK OF THE HOUSE:',
 '\n     Mr SPEAKER-ELECT:',
 '\n     [Subsequent authorised translation:',
 '\n     Therefore bear with me as I conclude this greeting with the words:',
 '\n     [Subsequent authorised translat

In [6]:
len(speakers_indices)

75097

In [7]:
speeches = list()
indices_start = list()
indices_end = list()
for speaker_number in range(0,len(speakers),1):
    if speaker_number == len(speakers) - 1:
        index_end = len(text_only) - 1
    else:
        index_start = speakers_indices[speaker_number][1]
        index_end = speakers_indices[speaker_number+1][0]
    indices_start = indices_start + [index_start]
    indices_end = indices_end + [index_end]
    speeches = speeches +  [text_only[index_start:index_end]]
print(len(speakers),len(speeches),len(indices_start),len(indices_end))

75097 75097 75097 75097


In [8]:
df_speeches = pd.DataFrame.from_dict({"Speaker": speakers, "start": indices_start, "end": indices_end, "Speech": speeches})

In [9]:
df_speeches.head(50)

,Speaker,start,end,Speech
0,\n JOHN CARTER (Senior Government Whip):,5166,6828,"I move, That Peter Wilfred Tapsell, the membe..."
1,\n Hon. K T WETERE (Western Maori):,6866,8004,E tu tautoko ana te motini kua whakatakotohia...
2,\n [Subsequent authorised translation:,8045,10526,I support the motion by the member for the Fa...
3,\n CLERK OF THE HOUSE:,10551,10616,Does the member propose to submit to the judg...
4,\n Hon. PETER TAPSELL (Eastern Maori):,10657,10788,I should like to express my sense of the hono...
5,\n CLERK OF THE HOUSE:,10813,10847,Are there any further proposals?
6,\n WINSTON PETERS (Tauranga):,10879,10980,"I move, That Robin McDowall Gray, the member ..."
7,\n Rt Hon. J B Bolger:,11005,11038,He's a member of the Executive.
8,\n WINSTON PETERS:,11059,14620,"He does not miss much, does he? First, I refe..."
9,\n Rt Hon. J B Bolger:,14645,14658,Keep it up!


In [10]:
df_speeches['important_speaker'] = [
    any([(speaker_word.isupper()) and (len(speaker_word) > 1) for speaker_word in speaker.split(" ")])
    for speaker in df_speeches['Speaker']
]

In [11]:
df_important_speeches = df_speeches[df_speeches.important_speaker]

In [124]:
exclude = set(string.punctuation)
df_speeches['speaker_name'] = [
    ' '.join([speaker_word for speaker_word in ''.join(ch for ch in speaker if ch not in exclude).split(" ")if((speaker_word.isupper()) and (len(speaker_word) > 2))])
    for speaker in df_speeches['Speaker']
]
df_speeches.speaker_name.unique()

array(['JOHN CARTER', 'WETERE', '', 'CLERK THE HOUSE', 'PETER TAPSELL',
       'WINSTON PETERS', 'TAU HENARE', 'ROBIN GRAY', 'SPEAKERELECT',
       'BOLGER', 'HELEN CLARK', 'JIM ANDERTON', 'MAX BRADFORD',
       'PAUL EAST', 'MIKE MOORE', 'SPEAKER', 'PAULINE GARDINER', 'DON',
       'JONATHAN HUNT', 'MICHAEL CULLEN', 'GEOFF BRAYBROOKE',
       'MICHAEL LAWS', 'HON DON MCKINNON', 'DAVID CAYGILL', 'BIRCH',
       'CLIVE MATTHEWSON', 'PHIL GOFF', 'ERIC ROY', 'SIMON UPTON',
       'PETE HODGSON', 'DEPUTY SPEAKER', 'ROB STOREY', 'JOHN BLINCOE',
       'NICK SMITH', 'MARGARET AUSTIN', 'GRAHAM', 'TREVOR MALLARD',
       'QUESTIONS THE DAY', 'JOHN ROBERTSON', 'LIANNE DALZIEL',
       'WYATT CREECH', 'ROGER MAXWELL', 'JENNY SHIPLEY', 'STEVE MAHAREY',
       'CHRIS FLETCHER', 'ELIZABETH TENNET', 'JOY', 'LOCKWOOD SMITH',
       'DOUG KIDD', 'RICK BARKER', 'BRIAN NEESON', 'GEORGE HAWKINS',
       'JOHN LUXTON', 'PETER GRESHAM', 'ROGER SOWRY', 'ALEC NEILL',
       'JIM SUTTON', 'JOHN FALLOON', 'JAC

In [127]:
test = pd.DataFrame.from_dict({"speaker_name": df_speeches['speaker_name']})
test = test.astype(str)
test1 = test.apply(lambda x: x.str.split().str[0])
test2 = test.apply(lambda x: x.str.split().str[1])
test2 = test2.fillna('')
test = test1 + ' ' + test2
test = test.astype(str)
df_speeches['speaker_name'] = test['speaker_name']
df_speeches.speaker_name.unique()

array(['JOHN CARTER', 'WETERE ', 'nan ', 'CLERK THE', 'PETER TAPSELL',
       'WINSTON PETERS', 'TAU HENARE', 'ROBIN GRAY', 'SPEAKERELECT ',
       'BOLGER ', 'HELEN CLARK', 'JIM ANDERTON', 'MAX BRADFORD',
       'PAUL EAST', 'MIKE MOORE', 'SPEAKER ', 'PAULINE GARDINER', 'DON ',
       'JONATHAN HUNT', 'MICHAEL CULLEN', 'GEOFF BRAYBROOKE',
       'MICHAEL LAWS', 'HON DON', 'DAVID CAYGILL', 'BIRCH ',
       'CLIVE MATTHEWSON', 'PHIL GOFF', 'ERIC ROY', 'SIMON UPTON',
       'PETE HODGSON', 'DEPUTY SPEAKER', 'ROB STOREY', 'JOHN BLINCOE',
       'NICK SMITH', 'MARGARET AUSTIN', 'GRAHAM ', 'TREVOR MALLARD',
       'QUESTIONS THE', 'JOHN ROBERTSON', 'LIANNE DALZIEL',
       'WYATT CREECH', 'ROGER MAXWELL', 'JENNY SHIPLEY', 'STEVE MAHAREY',
       'CHRIS FLETCHER', 'ELIZABETH TENNET', 'JOY ', 'LOCKWOOD SMITH',
       'DOUG KIDD', 'RICK BARKER', 'BRIAN NEESON', 'GEORGE HAWKINS',
       'JOHN LUXTON', 'PETER GRESHAM', 'ROGER SOWRY', 'ALEC NEILL',
       'JIM SUTTON', 'JOHN FALLOON', 'JACK ELDER

In [129]:
lab93 = "Sutherland Dalziel Moore Hodgson Matthewson Swain Mackey Yates Gallagher Barker Elder Keall Peck Tennet Dyson White Lange Hawkins King Clark Braybrooke Blincoe Hunt Duynhoven Northey Dunne Field Maharey Tizard Robertson Mallard Kelly Goff Caygill Cullen Carter Sutton Sinclair Burton Pettis O'Connor OConnor Austin Tapsell Sullivan Wetere"
nat93 = "Don Roy Revell Gray McCully Ryall Fletcher Carter Burdon Birch Hilt Kyd Laws Peter Meurant Rogers Anderson Smith Sowry Bolger Kidd Lee Luxton Cliffe Cooper Falloon Williamson Upton Shipley Gerard Marshall Graham East Richardson Simich Maxwell Bradford Smith Roger Storey O'Regan ORegan Creech Neeson Neill Gresham English Gardiner Quigley Banks"
stringtotest = "HELEN CLARK"
settotest = set(stringtotest.split())

test['name'] = [
    'SUTHERLAND (LAB)' if 'SUTHERLAND' in speaker_name
    else 'DALZIEL (LAB)' if 'DALZIEL' in speaker_name
    else 'MOORE (LAB)' if 'MOORE' in speaker_name
    else 'HODGSON (LAB)' if 'HODGSON' in speaker_name
    else 'MATTHEWSON (LAB)' if 'MATTHEWSON' in speaker_name
    else 'SWAIN (LAB)' if 'SWAIN' in speaker_name
    else 'MACKEY (LAB)' if 'MACKEY' in speaker_name
    else 'YATES (LAB)' if 'YATES' in speaker_name
    else 'GALLAGHER (LAB)' if 'GALLAGHER' in speaker_name
    else 'BARKER (LAB)' if 'BARKER' in speaker_name
    else 'ELDER (LAB)' if 'ELDER' in speaker_name
    else 'KEALL (LAB)' if 'KEALL' in speaker_name
    else 'PECK (LAB)' if 'PECK' in speaker_name
    else 'TENNET (LAB)' if 'TENNET' in speaker_name
    else 'DYSON (LAB)' if 'DYSON' in speaker_name
    else 'WHITE (LAB)' if 'WHITE' in speaker_name
    else 'LANGE (LAB)' if 'LANGE' in speaker_name
    else 'HAWKINS (LAB)' if 'HAWKINS' in speaker_name
    else 'KING (LAB)' if 'KING' in speaker_name
    else 'CLARK (LAB)' if 'CLARK' in speaker_name
    else 'BRAYBROOKE (LAB)' if 'BRAYBROOKE' in speaker_name
    else 'BLINCOE (LAB)' if 'BLINCOE' in speaker_name
    else 'HUNT (LAB)' if 'HUNT' in speaker_name
    else 'DUYNHOVEN (LAB)' if 'DUYNHOVEN' in speaker_name
    else 'NORTHEY (LAB)' if 'NORTHEY' in speaker_name
    else 'DUNNE (LAB)' if 'DUNNE' in speaker_name
    else 'PHILLIP FIELD (LAB)' if 'TAITO' in speaker_name
    else 'MAHAREY (LAB)' if 'MAHAREY' in speaker_name
    else 'TIZARD (LAB)' if 'TIZARD' in speaker_name
    else 'ROBERTSON (LAB)' if 'ROBERTSON' in speaker_name
    else 'MALLARD (LAB)' if 'MALLARD' in speaker_name
    else 'KELLY (LAB)' if 'KELLY' in speaker_name
    else 'GOFF (LAB)' if 'GOFF' in speaker_name
    else 'CAYGILL (LAB)' if 'CAYGILL' in speaker_name
    else 'CULLEN (LAB)' if 'CULLEN' in speaker_name
    else 'CARTER (LAB)' if 'CHRIS CARTER' in speaker_name
    else 'SUTTON (LAB)' if 'SUTTON' in speaker_name
    else 'SINCLAIR (LAB)' if 'SINCLAIR' in speaker_name
    else 'BURTON (LAB)' if 'BURTON' in speaker_name
    else 'PETTIS (LAB)' if 'PETTIS' in speaker_name
    else 'OCONNOR (LAB)' if 'OCONNOR' in speaker_name
    else 'AUSTIN (LAB)' if 'AUSTIN' in speaker_name
    else 'TAPSELL (LAB)' if 'TAPSELL' in speaker_name
    else 'SULLIVAN (LAB)' if 'SULLIVAN' in speaker_name
    else 'WETERE (LAB)' if 'WETERE' in speaker_name
    else 'MCKINNON (NAT)' if 'DON' in speaker_name
    else 'ROY (NAT)' if 'ROY' in speaker_name
    else 'REVELL (NAT)' if 'REVELL' in speaker_name
    else 'GRAY (NAT)' if 'GRAY' in speaker_name
    else 'MCCULLY (NAT)' if 'MURRAY' in speaker_name
    else 'RYALL (NAT)' if 'RYALL' in speaker_name
    else 'FLETCHER (NAT)' if 'FLETCHER' in speaker_name
    else 'JOHN CARTER (NAT)' if 'JOHN CARTER' in speaker_name
    else 'DAVID CARTER (NAT)' if 'CARTER' in speaker_name
    else 'BURDON (NAT)' if 'BURDON' in speaker_name
    else 'BIRCH (NAT)' if 'BIRCH' in speaker_name
    else 'HILT (NAT)' if 'HILT' in speaker_name
    else 'KYD (NAT)' if 'KYD' in speaker_name
    else 'LAWS (NAT)' if 'LAWS' in speaker_name
    else 'MEURANT (NAT)' if 'MEURANT' in speaker_name
    else 'ROGERS (NAT)' if 'ROGERS' in speaker_name
    else 'ANDERSON (NAT)' if 'ANDERSON' in speaker_name
    else 'NICK SMITH (NAT)' if 'SMITH' in speaker_name
    else 'SOWRY (NAT)' if 'SOWRY' in speaker_name
    else 'BOLGER (NAT)' if 'BOLGER' in speaker_name
    else 'KIDD (NAT)' if 'KIDD' in speaker_name
    else 'LEE (NAT)' if 'LEE' in speaker_name
    else 'CLIFFE (NAT)' if 'CLIFFE' in speaker_name
    else 'COOPER (NAT)' if 'COOPER' in speaker_name
    else 'FALLOON (NAT)' if 'FALLOON' in speaker_name
    else 'WILLIAMSON (NAT)' if 'WILLIAMSON' in speaker_name
    else 'UPTON (NAT)' if 'UPTON' in speaker_name
    else 'SHIPLEY (NAT)' if 'SHIPLEY' in speaker_name
    else 'GERARD (NAT)' if 'GERARD' in speaker_name
    else 'MARSHALL (NAT)' if 'MARSHALL' in speaker_name
    else 'GRAHAM (NAT)' if 'GRAHAM' in speaker_name
    else 'EAST (NAT)' if 'EAST' in speaker_name
    else 'RICHARDSON (NAT)' if 'RICHARDSON' in speaker_name
    else 'SIMICH (NAT)' if 'SIMICH' in speaker_name
    else 'MAXWELL (NAT)' if 'MAXWELL' in speaker_name
    else 'BRADFORD (NAT)' if 'BRADFORD' in speaker_name
    else 'LOCKWOOD SMITH (NAT)' if 'SMITH' in speaker_name
    else 'SOWRY (NAT)' if 'SOWRY' in speaker_name
    else 'STOREY (NAT)' if 'STOREY' in speaker_name
    else 'OREGAN (NAT)' if 'OREGAN' in speaker_name
    else 'CREECH (NAT)' if 'CREECH' in speaker_name
    else 'NEESON (NAT)' if 'NEESON' in speaker_name
    else 'NEILL (NAT)' if 'NEILL' in speaker_name
    else 'GRESHAM (NAT)' if 'GRESHAM' in speaker_name
    else 'ENGLISH (NAT)' if 'ENGLISH' in speaker_name
    else 'GARDINER (NAT)' if 'GARDINER' in speaker_name
    else 'QUIGLEY (NAT)' if 'QUIGLEY' in speaker_name
    else 'BANKS (NAT)' if 'BANKS' in speaker_name
    else ''
    for speaker_name in test['speaker_name']
]
df_speeches['name'] = test['name']
test.name.unique()

array(['JOHN CARTER (NAT)', 'WETERE (LAB)', '', 'TAPSELL (LAB)',
       'GRAY (NAT)', 'BOLGER (NAT)', 'CLARK (LAB)', 'BRADFORD (NAT)',
       'EAST (NAT)', 'MOORE (LAB)', 'GARDINER (NAT)', 'MCKINNON (NAT)',
       'HUNT (LAB)', 'CULLEN (LAB)', 'BRAYBROOKE (LAB)', 'LAWS (NAT)',
       'CAYGILL (LAB)', 'BIRCH (NAT)', 'MATTHEWSON (LAB)', 'GOFF (LAB)',
       'ROY (NAT)', 'UPTON (NAT)', 'HODGSON (LAB)', 'STOREY (NAT)',
       'BLINCOE (LAB)', 'NICK SMITH (NAT)', 'AUSTIN (LAB)',
       'GRAHAM (NAT)', 'MALLARD (LAB)', 'ROBERTSON (LAB)',
       'DALZIEL (LAB)', 'CREECH (NAT)', 'MAXWELL (NAT)', 'SHIPLEY (NAT)',
       'MAHAREY (LAB)', 'FLETCHER (NAT)', 'TENNET (LAB)', 'KIDD (NAT)',
       'BARKER (LAB)', 'NEESON (NAT)', 'HAWKINS (LAB)', 'GRESHAM (NAT)',
       'SOWRY (NAT)', 'NEILL (NAT)', 'SUTTON (LAB)', 'FALLOON (NAT)',
       'ELDER (LAB)', 'MARSHALL (NAT)', 'RYALL (NAT)', 'KING (LAB)',
       'ROGERS (NAT)', 'BANKS (NAT)', 'WILLIAMSON (NAT)', 'REVELL (NAT)',
       'PHILLIP FIELD (LAB)', 

In [130]:
labset93 = set(
    [
        word.upper()
        for word in lab93.split()
    ]
)
natset93 = set(
    [
        word.upper()
        for word in nat93.split()
    ]
)

if labset93.intersection(settotest) is not None:
    party='Labour'
    
party

'Labour'

In [131]:
stringtotest = "HELEN CLARK"
settotest = set(stringtotest.split())
len(labset93.intersection(settotest)) > 0

True

In [132]:
df_speeches['party'] = [
    'Labour' if '(LAB)' in name
    else 'National' if '(NAT)' in name
    else ""
    for name in df_speeches['name']
]

In [133]:
df_speeches[['speaker_name','name','party']]

,speaker_name,name,party
0,JOHN CARTER,JOHN CARTER (NAT),National
1,WETERE,WETERE (LAB),Labour
2,nan,,
3,CLERK THE,,
4,PETER TAPSELL,TAPSELL (LAB),Labour
...,...,...,...
75092,SPEAKER,,
75093,DON,MCKINNON (NAT),National
75094,SPEAKER,,
75095,SPEAKER,,


In [134]:
pd.options.display.max_colwidth = 500
pd.set_option('display.max_rows', 500)
df_important_speeches = df_speeches[df_speeches.important_speaker]
df_important_speeches.head(500)

,Speaker,start,end,Speech,important_speaker,speaker_name,party,name
0,\n JOHN CARTER (Senior Government Whip):,5166,6828,"I move, That Peter Wilfred Tapsell, the member for Eastern Maori, do take the chair of the House as Speaker. I am honoured and privileged to move this motion, and in doing so I am conscious of the historic significance of this nomination. We who have been elected to serve our country in this Parliament do so at a crucial time in our history. New Zealand has decided to change the method by which MPs are elected, and we in this House will be expected to reflect the new mood in the community i...",True,JOHN CARTER,National,JOHN CARTER (NAT)
1,\n Hon. K T WETERE (Western Maori):,6866,8004,"E tu tautoko ana te motini kua whakatakotohia ai i tenei o nga mema o te Tai Tokerau, ara, kia tu ai te mema o te Tai Rawhiti ki runga ki tenei turanga teitei. He tika tonu he tuatahi te honore kua homai nei kia ngai taua ki te iwi Maori, ara, mo Aotearoa katoa. Tuarua, kei te tautoko i nga hau e wha o te motu, tae atu ano ki te Whare nei. \n He mihi kau ake ki te iwi o Te Arawa---Te Arawa waka mai i Maketu ki Tongariro, otira, tena koutou mo te taonga nui kua utaina nei ki runga ki a k...",True,WETERE,Labour,WETERE (LAB)
3,\n CLERK OF THE HOUSE:,10551,10616,Does the member propose to submit to the judgment of the House?,True,CLERK THE,,
4,\n Hon. PETER TAPSELL (Eastern Maori):,10657,10788,I should like to express my sense of the honour proposed to be conferred on me and to submit myself to the judgment of the House.,True,PETER TAPSELL,Labour,TAPSELL (LAB)
5,\n CLERK OF THE HOUSE:,10813,10847,Are there any further proposals?,True,CLERK THE,,
6,\n WINSTON PETERS (Tauranga):,10879,10980,"I move, That Robin McDowall Gray, the member for Clutha, do take the chair of the House as Speaker.",True,WINSTON PETERS,,
8,\n WINSTON PETERS:,11059,14620,"He does not miss much, does he? First, I refer to the qualities of dignity and fairness that were cited by the then Senior Government Whip, the Senior Opposition Whip, the Prime Minister, the then Leader of the Opposition, and the leader of the NewLabour Party, when the member for Clutha was picked on 28 November 1990. Those comments are all recorded in Hansard. What has changed over the past 3 years in relation to those statements of adulation made 3 years ago? Do those qualities suddenly ...",True,WINSTON PETERS,,
10,\n WINSTON PETERS:,14679,14837,I will keep it up. That is why we have this problem here. The Prime Minister would not listen in the past 3 years; one would think that he might listen now.,True,WINSTON PETERS,,
12,\n WINSTON PETERS:,15077,15609,"Is it any wonder that longstanding supporters of Labour, NewLabour, and the Greens are struggling to see the reasons behind the support that has been given by those parties today---their championing of support for the National Party? My former colleagues in the National Party just cannot believe their luck. Just when the electorate had them on the hook, the leadership of \n 21 December 1993 \n \n \n the Alliance and the new leader of the Labour Party can hardly wait to get them off ...",True,WINSTON PETERS,,
13,\n TAU HENARE (Northern Maori):,15643,17266,"I second the motion moved by the member for Tauranga that the honourable member for Clutha take the chair as Speaker. It is with a certain amount of objection that I stand here today. It is not an objection to my colleague the honourable member for Tauranga, and it is certainly not an objection to being in these hallowed halls of power, but I am completely in objection to what I consider to be a return to the bad old days of two-party politics. \n On 6 November 1993, as members know, th...",True,TAU HENARE,,


In [135]:
df_important_speeches.to_csv("importantspeeches44.csv")

In [96]:
## https://towardsdatascience.com/topic-modelling-in-python-with-nltk-and-gensim-4ef03213cd21
parser = English()
def tokenize(text):
    lda_tokens = []
    tokens = parser(text)
    for token in tokens:
        if token.orth_.isspace():
            continue
        elif token.like_url:
            lda_tokens.append('URL')
        elif token.orth_.startswith('@'):
            lda_tokens.append('SCREEN_NAME')
        else:
            lda_tokens.append(token.lower_)
    return lda_tokens
def get_lemma(word):
    lemma = wn.morphy(word)
    if lemma is None:
        return word
    else:
        return lemma
    
def get_lemma2(word):
    return WordNetLemmatizer().lemmatize(word)
en_stop = set(nltk.corpus.stopwords.words('english'))
en_stop.update(['people','minister','ministers','would','house','member','government','committee','committees','office','offices','matter','matters','clause','clauses','agree','agreement','disagree','disagreement','opposition','member','point','order','answer','answers','question','speech','speaker','hear','department','members','objection','object','amend','amendment','amendments','vote','voting','votes','aye','no','think','thought','thinking','voted','ought','leader','prime','debate','debates','issue','issues','whether','support','supports','supporting','oppose','opposes','opposing','supported','opposed','zealand','process','legislation','legislator','legislators','party','parties','caucus','bring','bringing','brings','brought','labour','national','alliance','change','might','changes','changing','move','policy','submission','group','receive','report','submissions','groups','receives','reports','parliament'])
def prepare_text_for_lda(text):
    tokens = tokenize(text)
    tokens = [token for token in tokens if len(token) > 4]
    tokens = [token for token in tokens if token not in en_stop]
    tokens = [get_lemma(token) for token in tokens]
    return tokens

{'a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'agree',
 'agreement',
 'ain',
 'all',
 'alliance',
 'am',
 'amend',
 'amendment',
 'amendments',
 'an',
 'and',
 'answer',
 'answers',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'aye',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'bring',
 'bringing',
 'brings',
 'brought',
 'but',
 'by',
 'can',
 'caucus',
 'change',
 'changes',
 'changing',
 'clause',
 'clauses',
 'committee',
 'committees',
 'couldn',
 "couldn't",
 'd',
 'debate',
 'debates',
 'department',
 'did',
 'didn',
 "didn't",
 'disagree',
 'disagreement',
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'government',
 'group',
 'groups',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 'hear',
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'house',
 'how',
 'i',
 

In [111]:
text_data = []

for speech in df_important_speeches['Speech']:
        tokens = prepare_text_for_lda(speech)
        text_data.append(tokens)

In [112]:
len(text_data)

46751

In [113]:
dictionary = corpora.Dictionary(text_data)
corpus = [dictionary.doc2bow(text) for text in text_data]
pickle.dump(corpus, open('corpus.pkl', 'wb'))
dictionary.save('dictionary.gensim')

In [69]:
NUM_TOPICS = 30
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
ldamodel.save('model5.gensim')
topics = ldamodel.print_topics(num_words=10)

In [114]:
os.environ['MALLET_HOME'] = 'C:\\Users\\[PATH]\\Downloads\\mallet-2.0.8\\mallet-2.0.8'
mallet_path = 'C:\\Users\\[PATH]\\Downloads\\mallet-2.0.8\\mallet-2.0.8\\bin\\mallet'
ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=10, id2word=dictionary)
mtopics = ldamallet.print_topics(num_words=10)
for mtopic in mtopics:
    print(mtopic)

(0, '0.086*"health" + 0.035*"public" + 0.030*"services" + 0.022*"hospital" + 0.018*"service" + 0.016*"crown" + 0.016*"enterprise" + 0.015*"system" + 0.011*"advice" + 0.010*"regional"')
(1, '0.019*"person" + 0.017*"police" + 0.017*"court" + 0.012*"justice" + 0.010*"evidence" + 0.009*"force" + 0.008*"concern" + 0.008*"information" + 0.007*"inquiry" + 0.007*"crime"')
(2, '0.037*"country" + 0.016*"world" + 0.014*"unite" + 0.010*"nation" + 0.010*"years" + 0.010*"international" + 0.009*"great" + 0.008*"zealander" + 0.008*"trade" + 0.007*"place"')
(3, '0.039*"maori" + 0.024*"leave" + 0.022*"agree" + 0.016*"raise" + 0.016*"question" + 0.013*"paper" + 0.013*"standing" + 0.013*"seek" + 0.012*"affairs" + 0.012*"refer"')
(4, '0.036*"child" + 0.029*"school" + 0.028*"education" + 0.027*"family" + 0.019*"social" + 0.019*"woman" + 0.015*"young" + 0.014*"teacher" + 0.014*"programme" + 0.013*"housing"')
(5, '0.019*"company" + 0.019*"industry" + 0.015*"state" + 0.011*"public" + 0.011*"radio" + 0.010*"boa

In [148]:
df_important_speeches = df_important_speeches.reset_index(drop=True)
df_important_speeches.head(500)

,Speaker,start,end,Speech,important_speaker,speaker_name,party,category
0,\n JOHN CARTER (Senior Government Whip):,5166,6828,"I move, That Peter Wilfred Tapsell, the member for Eastern Maori, do take the chair of the House as Speaker. I am honoured and privileged to move this motion, and in doing so I am conscious of the historic significance of this nomination. We who have been elected to serve our country in this Parliament do so at a crucial time in our history. New Zealand has decided to change the method by which MPs are elected, and we in this House will be expected to reflect the new mood in the community i...",True,JOHN CARTER,Labour,
1,\n Hon. K T WETERE (Western Maori):,6866,8004,"E tu tautoko ana te motini kua whakatakotohia ai i tenei o nga mema o te Tai Tokerau, ara, kia tu ai te mema o te Tai Rawhiti ki runga ki tenei turanga teitei. He tika tonu he tuatahi te honore kua homai nei kia ngai taua ki te iwi Maori, ara, mo Aotearoa katoa. Tuarua, kei te tautoko i nga hau e wha o te motu, tae atu ano ki te Whare nei. \n He mihi kau ake ki te iwi o Te Arawa---Te Arawa waka mai i Maketu ki Tongariro, otira, tena koutou mo te taonga nui kua utaina nei ki runga ki a k...",True,WETERE,Labour,
2,\n CLERK OF THE HOUSE:,10551,10616,Does the member propose to submit to the judgment of the House?,True,CLERK OF THE HOUSE,,
3,\n Hon. PETER TAPSELL (Eastern Maori):,10657,10788,I should like to express my sense of the honour proposed to be conferred on me and to submit myself to the judgment of the House.,True,PETER TAPSELL,Labour,
4,\n CLERK OF THE HOUSE:,10813,10847,Are there any further proposals?,True,CLERK OF THE HOUSE,,
5,\n WINSTON PETERS (Tauranga):,10879,10980,"I move, That Robin McDowall Gray, the member for Clutha, do take the chair of the House as Speaker.",True,WINSTON PETERS,,
6,\n WINSTON PETERS:,11059,14620,"He does not miss much, does he? First, I refer to the qualities of dignity and fairness that were cited by the then Senior Government Whip, the Senior Opposition Whip, the Prime Minister, the then Leader of the Opposition, and the leader of the NewLabour Party, when the member for Clutha was picked on 28 November 1990. Those comments are all recorded in Hansard. What has changed over the past 3 years in relation to those statements of adulation made 3 years ago? Do those qualities suddenly ...",True,WINSTON PETERS,,
7,\n WINSTON PETERS:,14679,14837,I will keep it up. That is why we have this problem here. The Prime Minister would not listen in the past 3 years; one would think that he might listen now.,True,WINSTON PETERS,,
8,\n WINSTON PETERS:,15077,15609,"Is it any wonder that longstanding supporters of Labour, NewLabour, and the Greens are struggling to see the reasons behind the support that has been given by those parties today---their championing of support for the National Party? My former colleagues in the National Party just cannot believe their luck. Just when the electorate had them on the hook, the leadership of \n 21 December 1993 \n \n \n the Alliance and the new leader of the Labour Party can hardly wait to get them off ...",True,WINSTON PETERS,,
9,\n TAU HENARE (Northern Maori):,15643,17266,"I second the motion moved by the member for Tauranga that the honourable member for Clutha take the chair as Speaker. It is with a certain amount of objection that I stand here today. It is not an objection to my colleague the honourable member for Tauranga, and it is certainly not an objection to being in these hallowed halls of power, but I am completely in objection to what I consider to be a return to the bad old days of two-party politics. \n On 6 November 1993, as members know, th...",True,TAU HENARE,,Constitution


In [149]:
def format_topics_sentences(ldamodel=ldamallet, corpus=corpus, texts=df_important_speeches.dropna()['Speech']):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)


df_topic_sents_keywords = format_topics_sentences(ldamodel=ldamallet, corpus=corpus, texts=df_important_speeches.dropna()['Speech'])

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']


In [150]:
df_dominant_topic['category'] = [
    'Healthcare' if topic == 0
    else 'Justice' if topic == 1
    else 'International' if topic == 2
    else 'Nonspecific' if topic == 3
    else 'Education' if topic == 4
    else 'Fiscal' if topic == 5
    else 'Electoral' if topic == 6
    else 'Civic' if topic == 7
    else 'Monetary' if topic == 8
    else 'Nonspecific2' if topic == 9
    else ""
    for topic in df_dominant_topic['Dominant_Topic']
]
df_dominant_topic['party'] = df_important_speeches.dropna()['party']
df_dominant_topic['speaker_name'] = df_important_speeches.dropna()['speaker_name']

In [158]:
df_dominant_topic.head(70000)

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text,category,party,textblob,sentiment
0,0,2.0,0.1457,"country, world, unite, nation, years, international, great, zealander, trade, place","I move, That Peter Wilfred Tapsell, the member for Eastern Maori, do take the chair of the House as Speaker. I am honoured and privileged to move this motion, and in doing so I am conscious of the historic significance of this nomination. We who have been elected to serve our country in this Parliament do so at a crucial time in our history. New Zealand has decided to change the method by which MPs are elected, and we in this House will be expected to reflect the new mood in the community i...",International,Labour,"( , I, , m, o, v, e, ,, , T, h, a, t, , P, e, t, e, r, , W, i, l, f, r, e, d, , T, a, p, s, e, l, l, ,, , t, h, e, , m, e, m, b, e, r, , f, o, r, , E, a, s, t, e, r, n, , M, a, o, r, i, ,, , d, o, , t, a, k, e, , t, h, e, , c, h, a, i, r, , o, f, , t, h, e, , H, o, u, s, e, , a, s, ...)",0.111077
1,1,3.0,0.5448,"maori, leave, agree, raise, question, paper, standing, seek, affairs, refer","E tu tautoko ana te motini kua whakatakotohia ai i tenei o nga mema o te Tai Tokerau, ara, kia tu ai te mema o te Tai Rawhiti ki runga ki tenei turanga teitei. He tika tonu he tuatahi te honore kua homai nei kia ngai taua ki te iwi Maori, ara, mo Aotearoa katoa. Tuarua, kei te tautoko i nga hau e wha o te motu, tae atu ano ki te Whare nei. \n He mihi kau ake ki te iwi o Te Arawa---Te Arawa waka mai i Maketu ki Tongariro, otira, tena koutou mo te taonga nui kua utaina nei ki runga ki a k...",Nonspecific,Labour,"( , E, , t, u, , t, a, u, t, o, k, o, , a, n, a, , t, e, , m, o, t, i, n, i, , k, u, a, , w, h, a, k, a, t, a, k, o, t, o, h, i, a, , a, i, , i, , t, e, n, e, i, , o, , n, g, a, , m, e, m, a, , o, , t, e, , T, a, i, , T, o, k, e, r, a, u, ,, , a, r, a, ,, , k, i, a, , t, u, , a, ...)",0.000000
2,2,9.0,0.1090,"select, review, introduce, measure, relate, provisions, propose, accident, scheme, section",Does the member propose to submit to the judgment of the House?,Nonspecific2,,"( , D, o, e, s, , t, h, e, , m, e, m, b, e, r, , p, r, o, p, o, s, e, , t, o, , s, u, b, m, i, t, , t, o, , t, h, e, , j, u, d, g, m, e, n, t, , o, f, , t, h, e, , H, o, u, s, e, ?, )",0.000000
3,3,9.0,0.1267,"select, review, introduce, measure, relate, provisions, propose, accident, scheme, section",I should like to express my sense of the honour proposed to be conferred on me and to submit myself to the judgment of the House.,Nonspecific2,Labour,"( , I, , s, h, o, u, l, d, , l, i, k, e, , t, o, , e, x, p, r, e, s, s, , m, y, , s, e, n, s, e, , o, f, , t, h, e, , h, o, n, o, u, r, , p, r, o, p, o, s, e, d, , t, o, , b, e, , c, o, n, f, e, r, r, e, d, , o, n, , m, e, , a, n, d, , t, o, , s, u, b, m, i, t, , m, y, s, e, l, f, ...)",0.000000
4,4,9.0,0.1111,"select, review, introduce, measure, relate, provisions, propose, accident, scheme, section",Are there any further proposals?,Nonspecific2,,"( , A, r, e, , t, h, e, r, e, , a, n, y, , f, u, r, t, h, e, r, , p, r, o, p, o, s, a, l, s, ?, )",0.000000
...,...,...,...,...,...,...,...,...,...
46746,46746,3.0,0.1540,"maori, leave, agree, raise, question, paper, standing, seek, affairs, refer","No, this is not a valedictory---I know that Doug Graham was trying out my seat half an hour ago! It is time for a little bit of last housekeeping. I seek the leave of the House for Government, private, and local Bills on the Order Paper awaiting their second reading, with the exception of the New Zealand Courts Structure Bill and the Producer Board Acts Reform Bill, to be read a second time pro forma, and be referred to the appropriate select committee for consideration.",Nonspecific,National,"( , N, o, ,, , t, h, i, s, , i, s, , n, o, t, , a, , v, a, l, e, d, i, c, t, o, r, y, -, -, -, I, , k, n, o, w, , t, h, a, t, , D, o, u, g, , G, r, a, h, a, m, , w, a, s, , t, r, y, i, n, g, , o, u, t, , m, y, , s, e, a, t, , h, a, l, f

In [155]:
df_dominant_topic['textblob'] = [
    TextBlob(text)
    for text in df_dominant_topic['Text']
]

In [157]:
df_dominant_topic['sentiment'] = [
    textblob.sentiment.polarity
    for textblob in df_dominant_topic['textblob']
]

In [ ]:
df_dominant_topic.to_csv("dominanttopic.csv")
df_dominant_topic.head(70000)